In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torch.utils.data import DataLoader
from plots.gifs import trajectory_gif
from plots.plots import get_feature_history, plt_train_error, plt_norm_state, plt_norm_control, plt_classifier, feature_plot, plt_dataset
from models.training import Trainer, robTrainer
from models.neural_odes import NeuralODE, robNeuralODE
from models.resnets import ResNet
import pickle
import sys
import matplotlib.pyplot as plt
import imageio

from sklearn.datasets import make_moons, make_circles

#if trainin false, models will be loaded from file


In [2]:
hidden_dim, data_dim = 2, 2
T, num_steps = 5.0, 10  #T is the end time, num_steps are the amount of discretization steps for the ODE solver
dt = T/num_steps
turnpike = True
bound = 0.
fp = False
cross_entropy = True


training = True
num_epochs = 50 #number of optimization epochs for gradient decent

if turnpike:
    weight_decay = 0 if bound>0. else dt*0.01
else: 
    weight_decay = dt*0.01          #0.01 for fp, 0.1 else

In [3]:
anode = NeuralODE(device, data_dim, hidden_dim, augment_dim=0, non_linearity='tanh', 
                    architecture='outside', T=T, time_steps=num_steps, fixed_projector=fp, cross_entropy=cross_entropy)

optimizer_anode = torch.optim.Adam(anode.parameters(), lr=1e-3, weight_decay=weight_decay) #weight decay parameter modifies norm
trainer_anode = Trainer(anode, optimizer_anode, device, cross_entropy=cross_entropy, 
                        turnpike=turnpike, bound=bound, fixed_projector=fp, verbose = False)


anode_test = NeuralODE(device, data_dim, hidden_dim, augment_dim=0, non_linearity='tanh', 
                    architecture='outside', T=T, time_steps=num_steps, fixed_projector=fp, cross_entropy=cross_entropy)

optimizer_anode_test = torch.optim.Adam(anode_test.parameters(), lr=1e-3, weight_decay=weight_decay) #weight decay parameter modifies norm
trainer_anode_test = Trainer(anode_test, optimizer_anode_test, device, cross_entropy=cross_entropy, 
                        turnpike=turnpike, bound=bound, fixed_projector=fp, verbose = False)


In [4]:

rob_node = robNeuralODE(device, data_dim, hidden_dim, augment_dim=0, non_linearity='tanh', 
                            architecture='outside', T=T, time_steps=num_steps, fixed_projector=fp, cross_entropy=cross_entropy)


optimizer_rob_node = torch.optim.Adam(rob_node.parameters(), lr=1e-3, weight_decay=weight_decay) #weight decay parameter modifies norm
trainer_rob_node = robTrainer(rob_node, optimizer_rob_node, device, cross_entropy=cross_entropy, 
                        turnpike=turnpike, bound=bound, fixed_projector=fp, verbose = False)



                    

initializing self.adjoint_flow creates additional layers into rob_node. Does this effect the learning? Or can I just save/load only the important ones and it will be fine?

In [5]:
X, y = make_circles(noise=0.1, factor=0.1, random_state=1)
print('make circle', data_line)


import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

my_x = [np.array([[1.0,2],[3,4]]),np.array([[5.,6],[7,8]])] # a list of numpy arrays
my_y = [np.array([4.]), np.array([2.])] # another list of numpy arrays (targets)

tensor_x = torch.Tensor(my_x) # transform to torch tensor
tensor_y = torch.Tensor(my_y)

my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

with open('data.txt', 'rb') as fp:
    data_line, test = pickle.load(fp)
    print(data_line)

make circle [[-0.07009304  0.04274777]
 [ 0.05338575 -0.10321302]
 [-0.46597138 -1.06687743]
 [-0.1997198   0.07076349]
 [ 0.25612559  0.0220578 ]
 [-0.49776571  0.76955129]
 [-0.31914906 -0.84899624]
 [-0.13219642  0.04854691]
 [-0.02322423  0.10138229]
 [-0.03102069  0.06437639]
 [ 0.03522898 -0.13973031]
 [ 0.17181003 -0.92504802]
 [ 0.14889241  0.14202291]
 [ 0.19824756 -0.04897413]
 [ 0.12498214 -0.10121185]
 [-0.47960986 -0.7880243 ]
 [-0.90187751 -0.70895455]
 [ 0.74827659 -0.55976747]
 [-0.0979097  -0.11384725]
 [ 0.23161178  1.01205842]
 [ 0.81387506  0.22821311]
 [-0.24368754  0.02902427]
 [ 0.0956608  -0.10092505]
 [-0.03065257 -0.01692678]
 [-0.72651409  0.58630948]
 [-0.33930852 -1.0080984 ]
 [-0.02017866 -0.1459128 ]
 [-0.9539309  -0.33410313]
 [ 0.6927241  -0.59576417]
 [ 0.003546   -0.9098264 ]
 [ 0.02430112  0.23433603]
 [ 0.2535024  -0.05730649]
 [-0.09013857 -0.13100816]
 [-0.01960484 -0.09881616]
 [ 0.05990489  0.0301758 ]
 [ 0.31843503 -0.97937344]
 [ 0.92901189 -0

In [9]:
'''Train or load the models'''

#%%capture #this surpresses output
shuffle = False

#with open('data.txt', 'rb') as fp:
 #   data_line, test = pickle.load(fp)

X, y = make_circles(noise=0.1, factor=0.1, random_state=1)


dataloader = DataLoader(data_line, batch_size=64, shuffle=shuffle)
dataloader_viz = DataLoader(data_line, batch_size=128, shuffle=shuffle)


if training:
    trainer_anode.train(dataloader, num_epochs)
    trainer_anode_test.train(dataloader, num_epochs)

    trainer_rob_node.train(dataloader, num_epochs)

    torch.save(anode.state_dict(), 'anode.pth')
    torch.save(rob_node.state_dict(), 'rob_node.pth')
else:
    anode.load_state_dict(torch.load('anode.pth'))
    rob_node.load_state_dict(torch.load('rob_node.pth'))

ValueError: too many values to unpack (expected 2)

In [6]:
print(rob_node)
# print(anode)

robNeuralODE(
  (f_dynamics): Dynamics(
    (non_linearity): Tanh()
    (fc2_time): Sequential(
      (0): Linear(in_features=2, out_features=2, bias=True)
      (1): Linear(in_features=2, out_features=2, bias=True)
      (2): Linear(in_features=2, out_features=2, bias=True)
      (3): Linear(in_features=2, out_features=2, bias=True)
      (4): Linear(in_features=2, out_features=2, bias=True)
      (5): Linear(in_features=2, out_features=2, bias=True)
      (6): Linear(in_features=2, out_features=2, bias=True)
      (7): Linear(in_features=2, out_features=2, bias=True)
      (8): Linear(in_features=2, out_features=2, bias=True)
      (9): Linear(in_features=2, out_features=2, bias=True)
    )
  )
  (flow): Semiflow(
    (dynamics): Dynamics(
      (non_linearity): Tanh()
      (fc2_time): Sequential(
        (0): Linear(in_features=2, out_features=2, bias=True)
        (1): Linear(in_features=2, out_features=2, bias=True)
        (2): Linear(in_features=2, out_features=2, bias=True)
  

In [7]:


filename_base = '1traj'
filename_s = filename_base + '_s'
filename_r = filename_base + '_r'

plt_classifier(anode, num_steps=10, save_fig = '1generalization.pdf') 
plt_classifier(rob_node, num_steps=10, save_fig = '1rob_generalization.pdf') 

inputs, targets = next(iter(dataloader_viz))
print(sum(inputs))


# plt_classifier(rob_node, num_steps=10, save_fig = '1rob_generalization.pdf')
trajectory_gif(anode, inputs, targets, timesteps=num_steps, filename = filename_s +'.gif')
trajectory_gif(rob_node, inputs, targets, timesteps=num_steps, filename = filename_r + '.gif')

# plt.figure(1)
# plt.subplot(121)
# plt.imshow(imageio.imread(filename_s + '29.png'))
# plt.title('standard')
# plt.axis('off')
# plt.subplot(122)
# plt.imshow(imageio.imread(filename_r + '29.png'))
# plt.title('augmented robustness')
# plt.axis('off')

# plt.savefig('1comparison_' + filename_base + '.png',
#                     format='png', dpi=300, bbox_inches='tight')

# from IPython.display import Image
# Image(filename='1comparison_1traj.png') 

/home/np1/staff/woehrer/FAUbox/Python/borjan dynamical.systems/plots/plots.py:380: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = m(pre_)
/home/np1/staff/woehrer/FAUbox/Python/borjan dynamical.systems/plots/plots.py:380: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = m(pre_)


tensor([[-0.0701,  0.0427],
        [ 0.0534, -0.1032],
        [-0.4660, -1.0669],
        [-0.1997,  0.0708],
        [ 0.2561,  0.0221],
        [-0.4978,  0.7696],
        [-0.3191, -0.8490],
        [-0.1322,  0.0485],
        [-0.0232,  0.1014],
        [-0.0310,  0.0644],
        [ 0.0352, -0.1397],
        [ 0.1718, -0.9250],
        [ 0.1489,  0.1420],
        [ 0.1982, -0.0490],
        [ 0.1250, -0.1012],
        [-0.4796, -0.7880],
        [-0.9019, -0.7090],
        [ 0.7483, -0.5598],
        [-0.0979, -0.1138],
        [ 0.2316,  1.0121],
        [ 0.8139,  0.2282],
        [-0.2437,  0.0290],
        [ 0.0957, -0.1009],
        [-0.0307, -0.0169],
        [-0.7265,  0.5863],
        [-0.3393, -1.0081],
        [-0.0202, -0.1459],
        [-0.9539, -0.3341],
        [ 0.6927, -0.5958],
        [ 0.0035, -0.9098],
        [ 0.0243,  0.2343],
        [ 0.2535, -0.0573],
        [-0.0901, -0.1310],
        [-0.0196, -0.0988],
        [ 0.0599,  0.0302],
        [ 0.3184, -0

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [8]:
print(rob_node.state_dict)

print('anode')

for name, param in anode.named_parameters():
    if param.requires_grad:
        print(name, param.data)


print('\n','anode_test')


for name, param in anode_test.named_parameters():
    if param.requires_grad:
        print(name, param.data)

        
        
print('\n','rob_node')


for name, param in rob_node.named_parameters():
    if param.requires_grad:
        print(name, param.data)

<bound method Module.state_dict of robNeuralODE(
  (f_dynamics): Dynamics(
    (non_linearity): Tanh()
    (fc2_time): Sequential(
      (0): Linear(in_features=2, out_features=2, bias=True)
      (1): Linear(in_features=2, out_features=2, bias=True)
      (2): Linear(in_features=2, out_features=2, bias=True)
      (3): Linear(in_features=2, out_features=2, bias=True)
      (4): Linear(in_features=2, out_features=2, bias=True)
      (5): Linear(in_features=2, out_features=2, bias=True)
      (6): Linear(in_features=2, out_features=2, bias=True)
      (7): Linear(in_features=2, out_features=2, bias=True)
      (8): Linear(in_features=2, out_features=2, bias=True)
      (9): Linear(in_features=2, out_features=2, bias=True)
    )
  )
  (flow): Semiflow(
    (dynamics): Dynamics(
      (non_linearity): Tanh()
      (fc2_time): Sequential(
        (0): Linear(in_features=2, out_features=2, bias=True)
        (1): Linear(in_features=2, out_features=2, bias=True)
        (2): Linear(in_featur